In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
path = 'ImagesAttendance'
images = []
classNames = []
myList =os.listdir(path)
print(myList)
##
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)
#print(len(images))


def findEncodings(images):
    encodeList = []   
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #convert to RGB image
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList =f.readlines()
        nameList =[]
        #print(myDataList)
        for line in myDataList:
            entry= line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now =datetime.now()
            dtString=now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

#call function
encodeListKnown =findEncodings(images)
print('Encoding complete!')

#initial webcam:
cap = cv2.VideoCapture(0)
while True:
    success, img = cap.read()
    #imgS =cv2.resize(img,(0,0),None,0.25,0.25)
    imgS =cv2.resize(img,(50,50),interpolation=cv2.INTER_AREA)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB) #convert to RGB image
    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS,faceCurFrame)
    
    for encodeFace,faceLoc in zip(encodeCurFrame,faceCurFrame):
        matches=face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis =face_recognition.face_distance(encodeListKnown,encodeFace)
        matchIndex =np.argmin(faceDis)

        if (matches[matchIndex]):
            name =classNames[matchIndex].upper() #name
            #print(name)
            y1,x2,y2,x1=faceLoc
            y1,x2,y2,x1 =y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)#green
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y1-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendance(name)

    cv2.imshow('Webcam',img)
    cv2.waitKey(1)











['billgates.jpg', 'Elon Musk.jpg', 'Elon Test.jpg', 'Jackma.jpg']
['billgates', 'Elon Musk', 'Elon Test', 'Jackma']
Encoding complete!


KeyboardInterrupt: 